In [16]:
import pandas as pd
import numpy as np
import re

In [56]:
column_info = pd.read_csv('plan_summary.csv')
plan_data = pd.read_csv('../data_subsidy/Plan_Attributes_PUF_2016.csv')

/home/erichschulman/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (31,79,80,81,85,86,87,88,89,90,135,136,137,138) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [83]:
good_columns = column_info['Column_Name'][ (column_info['Missing_Values'] <= 3000) & (column_info['Missing_Values'] >= 0) & (column_info['Unique_Values'] >= 90) 
                  & (column_info['Unique_Values'] <= 370) ]
good_columns = list(good_columns)

#fix the set of columns to remove bad ones
bad_columns = ['SBCHavingSimplefractureCoinsurance', 
 'SBCHavingSimplefractureCopayment', 'SBCHavingSimplefractureDeductible', 'ServiceAreaID']
good_columns = list(set(good_columns) - set(bad_columns))


# https://stackoverflow.com/questions/4703390/how-to-extract-a-floating-number-from-a-string
clean_columns = plan_data[['ServiceAreaId','IssuerId']].copy()

for column in good_columns:
    #try to extract the text
    test_column = plan_data[column].copy()
    test_column = test_column.fillna(0)
    
    #if its just an int go with that
    int_column = test_column.apply(lambda x : float( (re.findall(r'^-?\d+(?:\.\d+)$',str(x)) +['0']) [0]) )
    if int_column.nunique() >= 3:
        clean_columns[column] = int_column
        
    #otherwise try something else
    dollar_column = test_column.apply(lambda x : int((re.findall('\\$(\\d+)',str(x) ) +['0'])[0]) )
    if dollar_column.nunique() >= 3:
        clean_columns[column] = dollar_column

print(clean_columns.columns)

Index(['ServiceAreaId', 'IssuerId', 'SBCHavingaBabyCopayment',
       'TEHBOutOfNetFamilyPerPersonMOOP', 'SBCHavingDiabetesCopayment',
       'TEHBInnTier1FamilyPerGroupMOOP', 'SBCHavingaBabyCoinsurance',
       'TEHBOutOfNetIndividualMOOP', 'TEHBCombInnOonFamilyPerGroupMOOP',
       'EHBPercentTotalPremium', 'SBCHavingDiabetesCoinsurance',
       'TEHBInnTier1IndividualMOOP', 'SBCHavingDiabetesDeductible',
       'TEHBInnTier1FamilyPerPersonMOOP',
       'OutOfServiceAreaCoverageDescription', 'TEHBCombInnOonIndividualMOOP',
       'SBCHavingaBabyDeductible', 'TEHBCombInnOonFamilyPerPersonMOOP',
       'TEHBOutOfNetFamilyPerGroupMOOP'],
      dtype='object')


In [88]:
clean_columns.describe()

,IssuerId,SBCHavingaBabyCopayment,TEHBOutOfNetFamilyPerPersonMOOP,SBCHavingDiabetesCopayment,TEHBInnTier1FamilyPerGroupMOOP,SBCHavingaBabyCoinsurance,TEHBOutOfNetIndividualMOOP,TEHBCombInnOonFamilyPerGroupMOOP,EHBPercentTotalPremium,SBCHavingDiabetesCoinsurance,TEHBInnTier1IndividualMOOP,SBCHavingDiabetesDeductible,TEHBInnTier1FamilyPerPersonMOOP,OutOfServiceAreaCoverageDescription,TEHBCombInnOonIndividualMOOP,SBCHavingaBabyDeductible,TEHBCombInnOonFamilyPerPersonMOOP,TEHBOutOfNetFamilyPerGroupMOOP
count,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000,26993.000000
mean,50966.640944,55.640499,3192.530471,134.833438,7151.497833,158.154744,3.335717,2218.278294,0.837373,65.326233,27.792465,73.176787,3531.508280,0.179676,3.109325,62.567369,1107.183085,6414.663061
std,25523.101290,146.803753,7763.868925,233.546216,5737.334815,286.061636,14.877219,8344.265129,0.363285,145.440168,121.458263,176.754650,2852.167722,4.090470,36.191283,182.389204,4185.963401,15250.054001
min,10046.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,31609.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.988000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,45786.000000,0.000000,0.000000,0.000000,8250.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.000000,1.000000,4000.000000,0.000000,0.000000,2.000000,0.000000,0.000000
75%,72034.000000,20.000000,0.000000,200.000000,13000.000000,220.000000,0.000000,0.000000,1.000000,40.000000,6.000000,5.000000,6500.000000,0.000000,0.000000,5.000000,0.000000,0.000000
max,99969.000000,950.000000,100000.000000,980.000000,13700.000000,990.000000,800.000000,93700.000000,1.000000,990.000000,950.000000,970.000000,6850.000000,100.000000,900.000000,900.000000,46850.000000,200000.000000
